# Imports

In [22]:
%load_ext autoreload
%autoreload 2

import pickle
import numpy as np
from datetime import datetime
import pandas as pd
import urllib.request
from fredapi import Fred

import matplotlib.pyplot as plt   
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

from consav.markov import tauchen, find_ergodic

from HANKSAMModel import HANKSAMModelClass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Setup

In [13]:
model = HANKSAMModelClass(name='baseline')

In [14]:
par = model.par
ss = model.ss
sim = model.sim
path = model.path

### Load Data

In [15]:
first = np.datetime64('1948-01')
last = np.datetime64(datetime.today().strftime("%Y-%m"))
data = pd.DataFrame(index=np.arange(first,last+1))

In [16]:
url = 'https://github.com/JeppeDruedahl/CPS-labor-market-flows/raw/main/Q.p'
filename = 'P.p'
urllib.request.urlretrieve(url,filename)

with open('P.p', 'rb') as f:
    P = pickle.load(f)

for k, v in P.items():
    
    data[f'l'] = v
    
    # calculate average transition event probability 
    data[k] = 100*(1.0-np.exp(-v))

In [17]:
mean_UE = np.mean(data['UE'])/100
mean_EU = np.mean(data['EU'])/100

par.lambda_u_ss = mean_UE
par.delta_ss = mean_EU/(1-mean_UE)

In [18]:
with open('data.p','wb') as f:
    pickle.dump(data,f)

# Calibration

## Find wage in steady state

In [10]:
par.UI_ratio_high = 0.83
par.UI_ratio_low = 0.54
par.UI_duration = 6.0

In [11]:
model.calibrate_to_C_drop(do_print=True)

guess on HtM_share of 0.01 causes diff drop - drop_target of 1.3741437030253492
guess on HtM_share of 0.3 causes diff drop - drop_target of -1.5620398502448616
guess on HtM_share of 0.1457209679325106 causes diff drop - drop_target of -0.000661484461000228
guess on HtM_share of 0.14565566609037586 causes diff drop - drop_target of -2.7998784979388347e-07
guess on HtM_share of 0.1456499378120454 causes diff drop - drop_target of 5.7720888918311175e-05
calibration done in 13.0 mins 58 secs


## Find lambda_u_s

In [14]:
par.UI_ratio_high = 0.83
par.UI_ratio_low = 0.54
par.UI_duration = 6.0

In [15]:
model.calibrate_to_lambda_u(do_print=True)

guess on lambda_eff of 0.2 causes diff drop - drop_target of -0.145668631071931
guess on lambda_eff of 0.4 causes diff drop - drop_target of 0.01433136892806347
guess on lambda_eff of 0.38208578883992006 causes diff drop - drop_target of 5.051514762044462e-15
guess on lambda_eff of 0.3820788784109759 causes diff drop - drop_target of -5.5283431806274486e-06
calibration done in 12.0 mins 16 secs
